Using a more of a scratch paper

In [ ]:
import spacy
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
import csv
from fuzzywuzzy import fuzz

nlp = spacy.load('en_core_web_sm')


In [5]:
business_resume = "resume_samples/Business_Resume.docx.pdf"
education_resume = "resume_samples/EducationResumedocx.pdf"
my_resume = "resume_samples/my_resume.pdf"

In [6]:
#fitz extractor
def extract_info_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    words = text.split()
    text = " ".join(words)
    return text

business_text = extract_info_from_pdf(business_resume)
education_text = extract_info_from_pdf(education_resume)
my_text = extract_info_from_pdf(my_resume)


In [7]:
business_text
# education_text
# my_text

'YOUR NAME first.last@selu.edu – 985-111-1111 EDUCATION Southeastern Louisiana University (SLU) Hammond, LA Bachelor of Arts in Marketing; Spanish minor May 2021 Major GPA: 3.50/4.00; Overall GPA: 3.65/4.00 RELEVANT EXPERIENCE Marketing Intern Baton Rouge, LA Louisiana Department of Labor May 2021 – July 2021 ● Screened 40+ applicants for open positions daily, paying attention to detail and clearly communicating job requirements to potential candidates ● Retrieved unemployment claims, collected confidential customer information, and conducted daily log entries diligently Chairman of Advertising/Public Relations Baton Rouge, LA American Red Cross, Fundraising Committee January 2020 – May 2020 ● Coordinated and supervised a 6-member committee and maximized committee potential by delegating to members’ strengths ● Provided personal outreach to various media outlets which led to first-time coverage from 4 outlets ● Wrote press releases and secured media publicity through radio, paper, and 

In [111]:
def extract_resume_info(text):

    doc = nlp(text)
    
    # name_pattern = r"([A-Z][a-z]+ [A-Z][a-z]+)"
    phone_pattern = r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
    email_pattern = r"[\w.-]+@[\w.-]+"
    website_pattern = r"((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"



    # name = [ent.text for ent in doc_first_two_lines.ents if ent.label_ == "PERSON"]
    name = doc[:2].text
    phone = re.search(phone_pattern, text).group(0) if re.search(phone_pattern, text) else ""
    email = re.search(email_pattern, text).group(0) if re.search(email_pattern, text) else ""
    website = re.search(website_pattern, text).group(0) if re.search(website_pattern, text) else ""
    
    return name, phone, email, website

In [110]:
website_pattern = r"((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
website = re.search(website_pattern, my_text).group(0) if re.search(website_pattern, my_text) else ""
website


'ucsd.edu'

In [118]:
business_output = extract_resume_info(business_text)
education_output = extract_resume_info(education_text)
my_output = extract_resume_info(my_text)

out = [business_output, education_output, my_output]
out

[('YOUR NAME', '985-111-1111', 'first.last@selu.edu', 'selu.edu'),
 ('Your Name', '985-555-5555', 'education@gmail.com', 'gmail.com'),
 ('Gihyeon Kwon', '(858) 753-3273', 'gikwon@ucsd.edu', 'ucsd.edu')]

In [38]:
#name
assert all([
    business_output[0] == "YOUR NAME", 
    education_output[0] == "Your Name", 
    my_output[0] == "Gihyeon Kwon"
    ]), "Name failed"
#number
assert all([
    business_output[1] == "985-111-1111", 
    education_output[1] == "985-555-5555",
    my_output[1] == "(858) 753-3273"
    ]), "number failed"
#email
assert all([
    business_output[2] == "first.last@selu.edu",
    education_output[2] == "education@gmail.com",
    my_output[2] == "gikwon@ucsd.edu"
    ]), "email failed"
#website
assert all([
    business_output[3] == "",
    education_output[3] == "linkedin.com/in/name",
    my_output[3] == "linkedin.com/in/gihyeonkwon"
    ]), "website failed"

NameError: name 'business_output' is not defined

In [35]:
business_text

'YOUR NAME first.last@selu.edu – 985-111-1111 EDUCATION Southeastern Louisiana University (SLU) Hammond, LA Bachelor of Arts in Marketing; Spanish minor May 2021 Major GPA: 3.50/4.00; Overall GPA: 3.65/4.00 RELEVANT EXPERIENCE Marketing Intern Baton Rouge, LA Louisiana Department of Labor May 2021 – July 2021 ● Screened 40+ applicants for open positions daily, paying attention to detail and clearly communicating job requirements to potential candidates ● Retrieved unemployment claims, collected confidential customer information, and conducted daily log entries diligently Chairman of Advertising/Public Relations Baton Rouge, LA American Red Cross, Fundraising Committee January 2020 – May 2020 ● Coordinated and supervised a 6-member committee and maximized committee potential by delegating to members’ strengths ● Provided personal outreach to various media outlets which led to first-time coverage from 4 outlets ● Wrote press releases and secured media publicity through radio, paper, and 

Found US univesrity csv file, gonna try to use it to extract college names from resume

In [70]:
#Option 1: broad
def find_colleges_in_text(text, college_names):
    found_colleges = None
    text_lower = text.lower() 
    
    for college in college_names:
        if college in text_lower:
            found_colleges.add(college)
            break
    return found_colleges

In [85]:
#Option 2: normalizing the text
def normalize_text(text):
    """
    Helper for extract_career_info function
    """
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.lower()

college_names_normalized = set()
with open('college_names', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        normalized_name = normalize_text(row[0].strip())  # Normalize each college name
        college_names_normalized.add(normalized_name)

In [89]:
def fuzzy_match_colleges(text, colleges, threshold=90):
    """
    Helper for extract_career_info function
    @param threshold : percentage of similarity to look for
    """
    matched_colleges = []
    normalized_text = normalize_text(text)
    for college in colleges:
        match_score = fuzz.ratio(normalized_text, college)

        if match_score > threshold:
            matched_colleges.append(college)
            
    return matched_colleges

In [90]:
fuzzy_match_colleges(my_text, college_names)

[]

In [69]:
find_colleges_in_text(business_text, college_names)

{'southeastern louisiana university'}

In [78]:
text = business_text
text = re.sub(r'[^\w\s]', '', text)
# text = re.sub(r'\s+', ' ', text) 
text

'YOUR NAME firstlastseluedu  9851111111 EDUCATION Southeastern Louisiana University SLU Hammond LA Bachelor of Arts in Marketing Spanish minor May 2021 Major GPA 350400 Overall GPA 365400 RELEVANT EXPERIENCE Marketing Intern Baton Rouge LA Louisiana Department of Labor May 2021  July 2021  Screened 40 applicants for open positions daily paying attention to detail and clearly communicating job requirements to potential candidates  Retrieved unemployment claims collected confidential customer information and conducted daily log entries diligently Chairman of AdvertisingPublic Relations Baton Rouge LA American Red Cross Fundraising Committee January 2020  May 2020  Coordinated and supervised a 6member committee and maximized committee potential by delegating to members strengths  Provided personal outreach to various media outlets which led to firsttime coverage from 4 outlets  Wrote press releases and secured media publicity through radio paper and internet sources positively representin

In [ ]:
text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
text = re.sub(r'\s+', ' ', text)  # Normalize whitespace